In [1]:
import re
import spacy
import textract
import os
from spacy.matcher import Matcher
import glob
import os
import random
#import resumes_parser
import pandas as pd
import json

import stanza
import spacy_stanza
# stanza.download('en')
# import resumes_parser

resumes_dict = {
    'resume_id':
    {                  #filename for now but to be replaced with unique id
        'name': ('first name', 'last name'),
        'email': ('email', 'position_tuple'),
        'phone': ('phone', 'position_tuple'),
        'dob': ('dob', 'position_tuple'),
        'address': ('address', 'position_tuple'),
        'skills': [('skill1', 'position_tuple1'), ('skill2', 'position_tuple2')],
    }                     
}

In [2]:
# list of files to be processed
#os.chdir('.')
file_list = glob.glob(os.path.join(os.getcwd(), "Fw__Sample_resumes_", "*.docx"))
# extracting text from docx files


In [3]:
file_list

['/home/kr/notebooks/qualifyde-resume-parsing/Fw__Sample_resumes_/Resume1_UI.docx',
 '/home/kr/notebooks/qualifyde-resume-parsing/Fw__Sample_resumes_/Resume2_Java_full_stack.docx',
 '/home/kr/notebooks/qualifyde-resume-parsing/Fw__Sample_resumes_/Resume4_Devops_engg.docx',
 '/home/kr/notebooks/qualifyde-resume-parsing/Fw__Sample_resumes_/Resume_11_Dotnet_Developer_.docx',
 '/home/kr/notebooks/qualifyde-resume-parsing/Fw__Sample_resumes_/Resume_12_backend_developer_.docx',
 '/home/kr/notebooks/qualifyde-resume-parsing/Fw__Sample_resumes_/Resume_6_data_engg.docx',
 '/home/kr/notebooks/qualifyde-resume-parsing/Fw__Sample_resumes_/Resume_9_cyber_security_.docx',
 '/home/kr/notebooks/qualifyde-resume-parsing/Fw__Sample_resumes_/resume_8_service_now.docx']

In [4]:
nlp = spacy.load('en_core_web_trf')

In [30]:
class parse:
    def __init__(self, file_list, nlp):
        self.file_list = file_list
        self.resumes = [textract.process(file_path).decode() for file_path in file_list]
        self.nlp = nlp

    def extract(self):
        ph_list, email_list, dob_list, names_list, location_list, skills_list = [], [], [], [], [], []
        for resume in self.resumes:
            items = re.finditer(r'\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}', resume)
            ph_span = [(resume[item.span()[0]:item.span()[1]], item.span()) for item in items]
            ph_list.append(ph_span)
            items = re.finditer(r'[\w\.-]+@[\w\.-]+', resume)
            em_span = [(resume[item.span()[0]:item.span()[1]], item.span()) for item in items]
            email_list.append(em_span)
            result2 = re.finditer(r"[\d]{1,2}-[\d]{1,2}-[\d]{4}", resume) # DD-MM-YYYY
            dob_span = [(resume[item.span()[0]:item.span()[1]], item.span()) for item in result2]
            dob_list.append(dob_span)
            doc = self.nlp(resume)
            list_ls, list_ns = [], []
            for ent in doc.ents:
                if ent.label_ == 'PERSON':
                    list_ns.append(ent.text)
                if ent.label_ in ['GPE', 'LOC']:
                    list_ls.append(ent.text)                  
            location_list.append(list_ls)
            names_list.append(list_ns)
            noun_chunks = doc.noun_chunks
            # tokens = [token.text for token in nlp_text if not token.is_stop]
            tokens = [token.text for token in doc if token.is_stop]
            
            data = pd.read_csv("skills.csv") 
            skills = list(data.columns.values)

            skillset = []
            for token in tokens:
                if token.lower() in skills:
                    skillset.append(token)

            for token in noun_chunks:
                token = token.text.lower().strip()
                if token in skills:
                    skillset.append(token)

            st = [i.capitalize() for i in set([i.lower() for i in skillset])]
            skills_list.append(st)
        return ph_list, email_list, dob_list, names_list, location_list, skills_list
    
    def dictify(self):
        dict_ = {}
        ph_list, email_list, dob_list, names_list, location_list, skills_list = self.extract()
        for idx, ph, email, dob, name, location, skills in zip(
            [file.split('.')[-2].split('/')[-1] for file in self.file_list], 
            ph_list, 
            email_list, 
            dob_list, 
            names_list, 
            location_list, 
            skills_list
        ):
            dict_[idx] = {}
            if len(dob):
                dict_[idx]['dob'] = dob[0]
            else:
                dict_[idx]['dob'] = (None, None)
            if len(name):
                dict_[idx]['name'] = name[0]
            else:
                dict_[idx]['name'] = (None, None)
            if len(email): 
                dict_[idx]['email'] = email[0]
            else:
                dict_[idx]['email'] = (None, None)
            if len(ph):
                dict_[idx]['phone'] = ph[0]
            else:
                dict_[idx]['phone'] = (None, None)
            if len(location):
                dict_[idx]['location'] = location[0]
            else:
                dict_[idx]['location'] = (None, None)
            dict_[idx]['skills'] = skills
        return dict_     

In [31]:
p = parse(file_list, nlp)

In [32]:
%%time
#ph_list, email_list, dob_list, names_list, location_list, skills_list, ents_ = p.extract()
resume_dict = p.dictify()

CPU times: user 3min 35s, sys: 12.5 s, total: 3min 48s
Wall time: 39.5 s


In [36]:
import json

with open('parsed_output.json', 'w') as f:
    json.dump(resume_dict, f)

In [45]:
print(ents_)

(Anil Kumar, 3+  , that 2.5 years, Excel, IBM India Private Limited, Saveetha Engineering college, Chennai, 2017, KFSH, Riyadh, #1, King Faisal Specialist Hospital and Research Centre, Riyadh, #2, Srikanth Vadlamudi, Indian, Telugu, English, Hindi, Tamil, Bangalore, Srikanth Vadlamudi)


In [19]:
for idx, name, location, ph_number, email, dob, skill in zip(res_index, names, locations, ph_numbers, emails, dobs, skills):
    print(idx, name)

Resume1_UI Salman Kaleem
Resume2_Java_full_stack Rajeshwar Rao
Resume4_Devops_engg Shivani Swaminathan
Resume_11_Dotnet_Developer_ Shivani Swaminathan
Resume_12_backend_developer_ Sid Chohan
Resume_6_data_engg Surya Kumer
Resume_9_cyber_security_ Asim Kalla
resume_8_service_now Srikanth Vadlamudi


In [ ]:
original_dict = {
    'AA_1_160322':{
    'name': names,
    'email': emails,
    'phone': ph_numbers,
    'dob': dobs,
    'location': locations,
    'skills': skills
    }
}

In [ ]:
original_dict

{'AA_1_160322': {'name': ['C',
   'L',
   'O',
   'U',
   'D',
   ' ',
   'A',
   'R',
   'C',
   'H',
   'I',
   'T',
   'E',
   'C',
   'T'],
  'email': [[('345@gmail.com', (24, 37))],
   [('12345@gmail.com', (24, 39))],
   [('122344@gmail.com', (12, 28))],
   [('abcd@gmail.com', (44, 58))],
   [('123451@gmail.com', (93, 109)),
    ('Srikanth50v51@gmail.com', (5294, 5317))],
   [('122690@gmail.com', (94, 110))],
   [('xxxxxi@gmail.com', (28, 44))],
   [('xyz@gmail.com', (46, 59)),
    ('eymemathew@gmail.com', (13099, 13119)),
    ('fyan@unr.edu', (13203, 13215)),
    ('SalmanKaleemM@Virtusa.com', (13282, 13307))]],
  'phone': [[('123-123-1234', (54, 66))],
   [('987-675-1234', (75, 87))],
   [('1234567', (54, 61)), ('997-1999', (3922, 3930))],
   [('(123) 456-5616', (62, 76))],
   [('1234567890', (59, 69))],
   [('123-466-7890', (116, 128))],
   [('123-456-7890', (15, 27)), ('707 205 6855', (46, 58))],
   [('123-456-7890', (25, 37))]],
  'dob': [[], [], [], [], [('03-10-1995', (5251,

In [ ]:
email_list = [email[0] for email in emails]
ph_list = [ph[0] for ph in ph_numbers]
names_list = [name for name in names]
loc_list = [loc for loc in locations]
skills_list = [skill for skill in skills]
dob_list = []
for dob in dobs:
    if dob:
        dob_list.append(dob[0])
    else:
        dob_list.append(('', (None, None)))

In [ ]:
zipped = zip(range(len(email_list)), names_list,email_list, ph_list, dob_list, loc_list, skills_list)

In [ ]:
original_dict = {}
for zip_ in zipped:
    original_dict[zip_[0]] = {}
    original_dict[zip_[0]]['name'] = zip_[1]
    original_dict[zip_[0]]['email'] = zip_[2]
    original_dict[zip_[0]]['phone'] = zip_[3]
    original_dict[zip_[0]]['dob'] = zip_[4]
    original_dict[zip_[0]]['location'] = zip_[5]
    original_dict[zip_[0]]['skills'] = zip_[6]

In [ ]:
original_dict

{0: {'name': 'C',
  'email': ('345@gmail.com', (24, 37)),
  'phone': ('123-123-1234', (54, 66)),
  'dob': ('', (None, None)),
  'location': 'India',
  'skills': ['Prototype',
   'Babel',
   'Js',
   'Cloud',
   'Mysql',
   'Sql server',
   'Apis',
   'Sdlc',
   'Xml',
   'Javascript',
   'Fulfillment',
   'Postgresql',
   'Json',
   'Scrum',
   'Flask',
   'Selenium',
   'Prototyping',
   'Windows',
   'Reports',
   'Oracle',
   'Test cases',
   'Banking',
   'Testing',
   'Workflows',
   'Process',
   'Sqlalchemy',
   'System',
   'Software development life cycle',
   'Analysis',
   'Pl/sql',
   'Transactions',
   'Design',
   'Java',
   'Html',
   'Jira',
   'Docker',
   'C',
   'Html5',
   'C++',
   'Jsp',
   'Css',
   'Servers',
   'Github',
   'Python',
   'Database',
   'Rest',
   'Sql',
   'Automation',
   'Soap',
   'Web services',
   'Mortgage',
   'Ui',
   'Aws']},
 1: {'name': 'L',
  'email': ('12345@gmail.com', (24, 39)),
  'phone': ('987-675-1234', (75, 87)),
  'dob': ('',

In [ ]:
with open('/media/anudeepadi/New Volume/Coding --python/extracted_data.json', 'w') as f:
    json.dump(original_dict, f)